# Descargar los datos de las spp y filtrar por un shp de Magallanes

In [1]:
import os
import geopandas as gpd
import requests 
import numpy as np
import pandas as pd
from tqdm import tqdm

ruta_actual = os.getcwd()
print(ruta_actual)

/Users/josepalmaduque/Desktop/Doctorado UACH/Python/clases_pipe


In [39]:
# Ruta al archivo shapefile
shp_file = "Users/josepalmaduque/Desktop/Doctorado UACH/Python/Regiones/Regional.shp" #MAC
shp_file = "D:\iCloudDrive\Desktop\Doctorado UACH\Python\Regiones\Regional.shp" #Windows
# Cargar el shapefile en un GeoDataFrame
gdf = gpd.read_file(shp_file)

print(gdf.head(10))


   objectid  cir_sena  codregion        area_km    st_area_sh    st_length_   
0      1084         1         15   16866.819844  1.886869e+10  7.505296e+05  \
1      1085         2          1   42284.567484  4.830637e+10  1.213713e+06   
2      1086         3          2  126071.431981  1.508452e+11  2.516112e+06   
3      1087        15         12  133053.142345  3.581316e+11  9.049830e+07   
4      1088        14         11  106703.377369  2.242743e+11  4.144481e+07   
5      1089         4          3   75661.248635  9.643906e+10  2.401741e+06   
6      1090         5          4   40575.901682  5.498082e+10  2.065933e+06   
7      1091         6          5   16322.970393  2.301475e+10  1.679609e+06   
8      1092         7         13   15392.030737  2.225204e+10  1.064253e+06   
9      1093        13         10   48408.371357  8.771834e+10  7.874158e+06   

                                      Region   
0               Región de Arica y Parinacota  \
1                         Región d

In [40]:
# Filtrar los datos para la Región de Magallanes
region_magallanes = gdf[gdf['codregion'] == 12]

In [6]:
# Ruta donde se guardará el nuevo shapefile
output_file = "/Users/josepalmaduque/Desktop/Doctorado UACH/Python/clases_pipe/spp/Magallanes.shp"

# Guardar los datos filtrados en un nuevo shapefile
region_magallanes.to_file(output_file)

# Leer el archivo shapefile de la región de Magallanes
magallanes = gpd.read_file('spp/Magallanes.shp')


### Generar un archivo con las ID de GBIF para poder descargar las spp

In [6]:
###Vamos a generar un archivo con el GBIFID de las spp

#df = pd.DataFrame()
df = pd.read_excel('..\..\Listado de especies Magallanes MMA.xlsx', sheet_name='Animales')

In [10]:
df.head()

,Ecosistema Magallanes,Reino,Clase,Orden,Familia,Nombre Científico,Nombre Común,Inventario Especies,GBIF,taxonid
0,Almirantazgo - Magallanes - Especies,Animalia,Aves,Procellariiformes,Diomedeidae,"Thalassarche melanophris (Temminck, 1828)",Thalassarche,--,https://www.gbif.org/species/2481410,2481410
1,Almirantazgo - Magallanes - Especies,Animalia,Aves,Falconiformes,Falconidae,"Falco sparverius cinnamominus Swainson, 1838",Falco,--,https://www.gbif.org/species/9685907,9685907
2,Almirantazgo - Magallanes - Especies,Animalia,Aves,Passeriformes,Furnariidae,"Cinclodes oustaleti Scott, 1900",Cinclodes,--,https://www.gbif.org/species/2485054,2485054
3,Almirantazgo - Magallanes - Especies,Animalia,Aves,Suliformes,Phalacrocoracidae,"Leucocarbo atriceps (King, 1828)",Leucocarbo,--,https://www.gbif.org/species/4848927,4848927
4,Almirantazgo - Magallanes - Especies,Animalia,Aves,Suliformes,Phalacrocoracidae,"Leucocarbo magellanicus (Gmelin, 1789)",Leucocarbo,--,https://www.gbif.org/species/2481856,2481856


In [11]:
df['taxonid'] = df.GBIF.apply(lambda x: x.split('/')[-1])

In [17]:
len(df.taxonid.unique())

2026

In [25]:
columna_unica = df['taxonid'].unique()

df_unicos = pd.DataFrame(columna_unica, columns = ['taxonid'])

df_unicos.to_csv('taxonid_especies.csv', index =False) 

## Descarga de datos de GBIF para chile

In [2]:
def request_gbif_api(taxonkey: int = 2440735):
    # Define GBIF API endpoint URL
    api_url = 'https://api.gbif.org/v1/occurrence/search/'

    # Define query parameters
    params = {
        'taxonKey': int(taxonkey),   # GBIF taxon key for whales
        'country': 'CL',     # ISO-3166 alpha-2 country code for Chile
        'year': "2015,2022", # Year
        'limit': 300,        # Maximum number of occurrences to retrieve per request
        'offset': 0          # Offset for pagination (starts at 0)
    }

    # Initialize empty list for occurrences
    occurrences = []
    # Loop over all pages of occurrences
    while True:
        # Send GET request to API endpoint with query parameters
        response = requests.get(api_url, params=params)
        results = response.json()['results']
        occurrences.extend(results)

        if len(results) < params['limit']:
            break

        # Increment offset for next page of occurrences
        params['offset'] += params['limit']


    df = pd.DataFrame({'gbifID': [k['gbifID'] for k in occurrences], 
                    'species': [k['species'] if 'species' in k.keys() else np.nan for k in occurrences],
                    'decimalLatitude': [k['decimalLatitude'] if 'decimalLatitude' in k.keys() else np.nan for k in occurrences],
                    'decimalLongitude': [k['decimalLongitude'] if 'decimalLongitude' in k.keys() else np.nan for k in occurrences],
                    'phylumKey': [k['phylumKey'] if 'phylumKey' in k.keys() else np.nan for k in occurrences],
                    'classKey': [k['classKey'] if 'classKey' in k.keys() else np.nan for k in occurrences],
                    'orderKey': [k['orderKey'] if 'orderKey' in k.keys() else np.nan for k in occurrences],
                    'familyKey': [k['familyKey'] if 'familyKey' in k.keys() else np.nan for k in occurrences],
                    'geodeticDatum': [k['geodeticDatum'] if 'geodeticDatum' in k.keys() else np.nan for k in occurrences],   
                    'eventDate': [k['eventDate'] if 'eventDate' in k.keys() else np.nan for k in occurrences]})

    df = df.dropna()
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['decimalLongitude'], df['decimalLatitude']), crs='EPSG:4326')

    return gdf, occurrences

In [4]:
def descarga_muchas_especies(especies: list): #Para MAC
    archivos = !ls spp 
    
    for especie in tqdm(especies):

        if f'datos_{especie}.csv' in archivos:
            print(f'ya tenemos descargado {especie}')
        else:
            print(f'descargando {especie}')
            _df, _ = request_gbif_api(taxonkey=especie)
            _df.to_csv(f'spp/datos_{especie}.csv', index=False)


In [3]:
def descarga_muchas_especies(especies: list): #Para Windows
    archivos = os.listdir('spp')
    
    for especie in tqdm(especies):

        if f'datos_{especie}.csv' in archivos:
            print(f'ya tenemos descargado {especie}')
        else:
            print(f'descargando {especie}')
            _df, _ = request_gbif_api(taxonkey=especie)
            _df.to_csv(os.path.join('spp', f'datos_{especie}.csv'), index=False)

In [2]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import requests
from tqdm import tqdm
import concurrent.futures

def request_gbif_api(taxonkey: int = 2440735, output_file: str = None):
    # Define GBIF API endpoint URL
    api_url = 'https://api.gbif.org/v1/occurrence/search/'

    # Define query parameters
    params = {
        'taxonKey': int(taxonkey),   # GBIF taxon key for whales
        'country': 'CL',     # ISO-3166 alpha-2 country code for Chile
        'year': "2015,2022", # Year
        'limit': 300,        # Maximum number of occurrences to retrieve per request
        'offset': 0          # Offset for pagination (starts at 0)
    }

    # Initialize empty list for occurrences
    occurrences = []

    # Loop over all pages of occurrences
    while True:
        # Send GET request to API endpoint with query parameters
        response = requests.get(api_url, params=params)
        results = response.json()['results']
        occurrences.extend(results)

        if len(results) < params['limit']:
            break

        # Increment offset for next page of occurrences
        params['offset'] += params['limit']

    df = pd.DataFrame({'gbifID': [k['gbifID'] for k in occurrences], 
                    'species': [k['species'] if 'species' in k.keys() else np.nan for k in occurrences],
                    'decimalLatitude': [k['decimalLatitude'] if 'decimalLatitude' in k.keys() else np.nan for k in occurrences],
                    'decimalLongitude': [k['decimalLongitude'] if 'decimalLongitude' in k.keys() else np.nan for k in occurrences],
                    'phylumKey': [k['phylumKey'] if 'phylumKey' in k.keys() else np.nan for k in occurrences],
                    'classKey': [k['classKey'] if 'classKey' in k.keys() else np.nan for k in occurrences],
                    'orderKey': [k['orderKey'] if 'orderKey' in k.keys() else np.nan for k in occurrences],
                    'familyKey': [k['familyKey'] if 'familyKey' in k.keys() else np.nan for k in occurrences],
                    'geodeticDatum': [k['geodeticDatum'] if 'geodeticDatum' in k.keys() else np.nan for k in occurrences],   
                    'eventDate': [k['eventDate'] if 'eventDate' in k.keys() else np.nan for k in occurrences]})

    df = df.dropna()
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['decimalLongitude'], df['decimalLatitude']), crs='EPSG:4326')

    if output_file:
        gdf.to_csv(output_file, index=False)

    return gdf, occurrences



In [3]:
def descarga_muchas_especies(especies):
    num_especies = len(especies)

    def descargar_especie(especie):
        archivos = os.listdir('spp')
        if f'datos_{especie}.csv' in archivos:
            print(f'Ya tenemos descargado {especie}')
        else:
            print(f'Descargando {especie}')
            df, _ = request_gbif_api(taxonkey=especie)
            df.to_csv(os.path.join('spp', f'datos_{especie}.csv'), index=False)

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        list(tqdm(executor.map(descargar_especie, especies), total=num_especies))



In [4]:
especies = pd.read_csv('taxonid_especies.csv')
especies = list(especies.taxonid.unique())
len(especies)

2026

In [5]:
descarga_muchas_especies(especies)

Ya tenemos descargado 2481410
Ya tenemos descargado 9685907
Ya tenemos descargado 2485054
Ya tenemos descargado 4848927
Ya tenemos descargado 2498234
Ya tenemos descargado 2481856
Ya tenemos descargado 2480989
Ya tenemos descargado 5231103
Ya tenemos descargado 2483725
Ya tenemos descargado 5232428
Ya tenemos descargado 2481432
Ya tenemos descargado 2481907
Ya tenemos descargado 2481515
Ya tenemos descargado 5232424
Ya tenemos descargado 5229231
Ya tenemos descargado 2481930
Ya tenemos descargado 2481610
Ya tenemos descargado 2485058


  4%|█▌                                      | 82/2026 [00:00<00:02, 802.24it/s]

Ya tenemos descargado 2484424
Ya tenemos descargado 2481065
Ya tenemos descargado 5231073
Ya tenemos descargado 2482738
Ya tenemos descargado 2480262
Ya tenemos descargado 2483790
Ya tenemos descargado 2485042
Ya tenemos descargado 5231459
Ya tenemos descargado 5789239
Ya tenemos descargado 2481495
Ya tenemos descargado 2480786
Ya tenemos descargado 2480265
Ya tenemos descargado 5229651
Ya tenemos descargado 2498236
Ya tenemos descargado 2485838
Ya tenemos descargado 2498294
Ya tenemos descargado 2481408
Ya tenemos descargado 2481173
Ya tenemos descargado 2490775
Ya tenemos descargado 5229770
Ya tenemos descargado 2481770
Ya tenemos descargado 5229094
Ya tenemos descargado 5229385
Ya tenemos descargado 2485069
Ya tenemos descargado 2482688
Ya tenemos descargado 2481428
Ya tenemos descargado 4352370
Ya tenemos descargado 2478578
Ya tenemos descargado 2498065
Ya tenemos descargado 2491010
Ya tenemos descargado 5232423
Ya tenemos descargado 2475466
Ya tenemos descargado 2481829
Ya tenemos

 12%|████▌                                  | 237/2026 [00:00<00:02, 716.98it/s]

Ya tenemos descargado 2319837
Ya tenemos descargado 6788988
Ya tenemos descargado 8288896
Ya tenemos descargado 4316038
Ya tenemos descargado 4315941
Ya tenemos descargado 4373710
Ya tenemos descargado 2281713
Ya tenemos descargado 2306037
Ya tenemos descargado 5726206
Ya tenemos descargado 2306767
Ya tenemos descargado 2485881
Ya tenemos descargado 2293457
Ya tenemos descargado 2480254
Ya tenemos descargado 2481110
Ya tenemos descargado 2482063
Ya tenemos descargado 2474382
Ya tenemos descargado 2498138
Ya tenemos descargado 2474388
Ya tenemos descargado 2485888
Ya tenemos descargado 2480287
Ya tenemos descargado 2491869
Ya tenemos descargado 2482087
Ya tenemos descargado 2485189
Ya tenemos descargado 5232427
Ya tenemos descargado 2481721
Ya tenemos descargado 2485141
Ya tenemos descargado 5229389
Ya tenemos descargado 2495358
Ya tenemos descargado 5231190
Ya tenemos descargado 5230075
Ya tenemos descargado 2481006
Ya tenemos descargado 5229170
Ya tenemos descargado 2481748
Ya tenemos

...

Descargando 10760815
Descargando 2073986
Descargando 5216368
Descargando 2434870
Descargando 6519590
Descargando 8215487
Descargando 2393056
Descargando 2481765
Descargando 2481744
Descargando 5229688
Descargando 2489160
Descargando 2486493


 49%|██████████████████▍                   | 983/2026 [21:56<2:18:32,  7.97s/it]

Descargando 2483719


 49%|███████████████████▌                    | 994/2026 [22:29<23:20,  1.36s/it]


KeyboardInterrupt: 

# Generar las grillas de 1kmx1km para mapear